In [ ]:
def _calc_eclipse_duration(self, P, R_star, R_planet, a, b, i):
        """
        TODO: Add units to this

        Parameters
        ----------
            P: float
                Orbital period of the exoplanet.
            R_star: float
                Stellar radius
            R_planet: float
                Planetary radius
            a: float
                Semi-major axis of the exoplanet orbit
            b: float
                Impact parameter
            i: float
                Inclination

        """
        k = R_planet/R_star
        transit_duration = (P/np.pi) * (np.arcsin((R_star/a) * (((1+k)**2-(b)**2)**1/2) / np.sin(i)))
        return transit_duration
    
    def _get_eclipse_system_params(self, obj_name, ra=None, dec=None):
        """Queries the NASA Exoplanet Archive for system parameters used in eclipse duration calculation.

        Parameters
        ----------
            obj_name: str
                The name of the exoplanet object.
            ra: float (Optional)
                The right ascension of the object to observe in the sky (most likely a planet or star).
            dec: float (Optional)
                The declination of the object to observe in the sky (most likely a planet or star).
            
        Returns
        -------
        """
        nea_data = self._query_nasa_exoplanet_archive(obj_name, select_query="pl_rade,st_rad,pl_orbsmax,pl_imppar,pl_orbincl")
        params = ["pl_rade", "st_rad", "pl_orbsmax", "pl_imppar", "pl_orbincl"]
        return_data = {}
        for param in params:
            for val in nea_data[param]:
                # If it is real value and not nan
                if not(np.isnan(val)):
                    val_to_store = val
                    if isinstance(val, Quantity) and hasattr(val, 'mask'):
                        # If the value is masked, just store value
                        val_to_store = val.value
                    if param == "pl_rade":
                        # Convert earth radii to km
                        val_to_store = (val_to_store * R_earth).to(u.km)
                    elif param == "st_rad":
                        # Convert solar radii to km
                        val_to_store = (val_to_store * R_sun).to(u.km)
                        # return_data[param] = val_km.value
                    elif param == "pl_orbsmax":
                        val_to_store = (val_to_store * au).to(u.km)
                    return_data[param] = val_to_store
                    # Break the param value loop once a real value has been found
                    break
        return return_data